# Доверительные интервалы для двух долей 

In [7]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

## Загрузка данных

In [8]:
data = pd.read_csv('banner_click_stat.txt', header = None, sep = '\t')
data.columns = ['banner_a', 'banner_b']

In [9]:
data.head() #клик или нет -по каждому из баннерах

,banner_a,banner_b
0,0,0
1,1,1
2,0,0
3,0,0
4,0,0


In [10]:
data.describe() #кол-ву людей одинаковому показали эти баннеры 

,banner_a,banner_b
count,1000.000000,1000.000000
mean,0.037000,0.053000
std,0.188856,0.224146
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,1.000000,1.000000


## Интервальные оценки долей

$$\frac1{ 1 + \frac{z^2}{n} } \left( \hat{p} + \frac{z^2}{2n} \pm z \sqrt{ \frac{ \hat{p}\left(1-\hat{p}\right)}{n} + \frac{z^2}{4n^2} } \right), \;\; z \equiv z_{1-\frac{\alpha}{2}}$$ 

In [15]:
conf_interval_banner_a = proportion_confint(sum(data.banner_a), 
                                            data.shape[0],
                                            method = 'wilson')
conf_interval_banner_b = proportion_confint(sum(data.banner_b), 
                                            data.shape[0],
                                            method = 'wilson')

 Давайте для начала интервально оценим баннеры. Мы видим, что доля кликов (или доля успехов) у нас относительно небольшая, поэтому для того, чтобы построить интервальную оценку доли, нам подойдет метод Уилсона. Из предыдущих лекции вы помните, что он хорошо работает в случае крайних значений на вероятность. Итак, давайте построим доверительные интервалы, делаем мы это с помощью метода proportion_confint из библиотеки StatsModels, и сразу же выведем их на экран. Видим, что мы получили интервалы. Интервалы достаточно узкие, но тем не менее они пересекаются. Это не очень удобно, потому что если бы один интервал лежал, например, полностью левее, чем другой, тогда нам было бы легко понять, что один баннер действительно лучше. А сейчас они пересекаются и не очень понятно, как их сравнить. 

In [16]:
print ('interval for banner a [%f, %f]' % conf_interval_banner_a)
print ('interval for banner b [%f, %f]' % conf_interval_banner_b)

interval for banner a [0.026961, 0.050582]
interval for banner b [0.040747, 0.068675]


### Как их сравнить?

## Доверительный интервал для разности долей (независимые выборки)

   | $X_1$ | $X_2$  
  ------------- | -------------|
  1  | a | b 
  0  | c | d 
  $\sum$ | $n_1$| $n_2$
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

 Для того, чтобы их сравнить, давайте построим доверительный интервал на разность двух долей. Табличка: давайте выпишем статистику по кликам на первый баннер и на второй, соответственно, посчитаем, сколько у нас было кликов — сколько было 1 в выборке, и сколько было не-кликов, или сколько было 0. На основе этого давайте рассчитаем статистики p1 и p2 и по формуле ниже оценим интервально разность долей. Давайте эту формулу реализуем, она достаточно простая. Смотрите, на реализации здесь всё просто. Сначала мы с вами честно рассчитываем все те слагаемые, которые входят в формулу, а дальше просто по формуле выписываем левую и правую границы интервала. Вот давайте это запустим и выведем на экран получившийся интервал. Ну, что можно сказать? Мы видим, что относительно 0 наш интервал более сильно сдвинут влево, большая часть его лежит все-таки в отрицательной плоскости, но тем не менее 0 внутри и не очень понятно, действительно ли старый баннер хуже, чем новый. Давайте рассмотрим следующую ситуацию. Предположим, что мы показывали баннер одним и тем же людям, то есть одни и те же люди оценивали как баннер A, старый баннер, так и баннер B. В этом случае мы с вами можем сказать, что речь идет уже о связанных выборках, они уже не являются независимыми. В этом случае мы можем использовать другую оценку разности долей. Для того, чтобы такую оценку получить, сначала нам нужно выписать таблицу сопряженности. 

In [17]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [18]:
print ("confidence interval: [%f, %f]" % proportions_confint_diff_ind(data.banner_a, data.banner_b))

confidence interval: [-0.034157, 0.002157]


Это очень простая таблица, у которой по строкам и по столбцам написаны значения наших случайных величин, в данном случае это 1 и 0, и в ячейках написано количество наблюдений, соответствующих данным значениям величин. Для того, чтобы получить доверительный интервал, нам с вами нужно по формулам рассчитать p1 и p2, дальше рассчитать их разность и подставить всё это для того, чтобы получить левую и правую границу. Давайте эту функцию реализуем, здесь всё довольно просто и также похоже на предыдущий случай. Сначала мы с вами оцениваем z, дальше оцениваем f и g и подставляем их в формулу для получения интервалов. Итак, давайте эту функцию построим и оценим интервал. Отлично, смотрите: здесь мы видим, что весь наш интервал лежит полностью левее нуля. Из этого можно сделать вывод, что действительно наш новый баннер лучше, чем старый, и выбрать нужно именно его. На этом мы с вами заканчиваем. Мы научились строить доверительный интервал на разность двух долей, а в следующем видео вы познакомитесь с новой технологией для оценки доверительных интервалов. Технология называется bootstrap. 

## Доверительный интервал для разности долей (связанные выборки)

  $X_1$ \ $X_2$ | 1| 0 | $\sum$
  ------------- | -------------|
  1  | e | f | e + f
  0  | g | h | g + h
  $\sum$ | e + g| f + h | n  
  
$$ \hat{p}_1 = \frac{e + f}{n}$$

$$ \hat{p}_2 = \frac{e + g}{n}$$

$$ \hat{p}_1 - \hat{p}_2 = \frac{f - g}{n}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\;  \frac{f - g}{n} \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{f + g}{n^2} - \frac{(f - g)^2}{n^3}}$$

In [ ]:
def proportions_confint_diff_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [ ]:
print "confidence interval: [%f, %f]" % proportions_confint_diff_rel(data.banner_a, data.banner_b)

In [ ]:
Quiz 2
